In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Constantes globales
R = 8.314  # J/(mol·K)
P = 1.01325  # bar
V_total = 200  # m^3
T0 = 298.15  # K
T_ext = 1100.0  # K (Temperatura del reactor)
Ftot0 = 6.5 * 10  # mol/s

# Pesos moleculares (g/mol)
MW = {
    "C": 12.01,
    "CO2": 44.01,
    "CO": 28.01,
    "H2O": 18.02,
    "H2": 2.016,
    "CH4": 16.04,
}

# Parámetros de cada reacción
data = {
    "Reacción 1": {
        "A": 3.1e6, "n": 0.38, "Ea": 215e3,
        "P_init": 0.738 / 157.67,
        "species": ["C", "CO2", "CO"],
        "stoich": [-1, -1, +2],
        "MW": [MW["C"], MW["CO2"], MW["CO"]],
    },
    "Reacción 2": {
        "A": 123e6, "n": 0.75, "Ea": 198e3,
        "P_init": 12.08 / 157.67,
        "species": ["C", "H2O", "CO", "H2"],
        "stoich": [-1, 0, +1, -1],
        "MW": [MW["C"], MW["H2O"], MW["CO"], MW["H2"]],
    },
    "Reacción 3": {
        "A": 4.189e-3, "n": 1.2, "Ea": 19210,
        "P_init": 1.2 / 157.67,
        "species": ["C", "H2", "CH4"],
        "stoich": [-1, -2, +1],
        "MW": [MW["C"], MW["H2"], MW["CH4"]],
    }
}

def reaction_rate(params, T, X):
    P_current = P * (params["P_init"] * max(1 - X, 1e-6)) * 0.08314 * T
    k = params["A"] * np.exp(-params["Ea"] / (R * max(T, 1e-6)))
    return k * (P_current ** params["n"])

def reactor_equations(V, y, params):
    FA = y[0]  # Flujo del reactivo principal (C)
    X = (FA0 - FA) / FA0
    r = reaction_rate(params, y[-1], X)
    dydV = [nu * r for nu in params["stoich"]]
    
    # Balance de energía: transferencia de calor con la camisa térmica
    Cp_medio = 15  # J/(mol·K), valor aproximado
    U = 1  # J/(s·m^2·K)
    a = 40  # m^2/m^3
    F_total = sum(y[:-1])  # Flujo total sin temperatura
    dT_dV = (-U * a * (y[-1] - T_ext)) / (F_total * Cp_medio)
    dydV.append(dT_dV)
    
    return dydV

def analyze_reaction(name, params):
    global FA0
    FA0 = 0.5 * Ftot0  # Flujo inicial de C
    y0 = [FA0 * 0.2 if sp != "C" else FA0 for sp in params["species"]] + [T0]
    sol = solve_ivp(lambda V, y: reactor_equations(V, y, params), (0, V_total), y0, dense_output=True)
    final_flows = sol.y[:, -1]
    final_masses = [flow * MW / 1000 for flow, MW in zip(final_flows[:-1], params["MW"])]
    print(f"{name} - Concentraciones finales (kg/s):")
    for sp, mass in zip(params["species"], final_masses):
        print(f"  {sp}: {mass:.4f} kg/s")
    return sol

# Analizar cada reacción por separado
for reaction_name, reaction_params in data.items():
    analyze_reaction(reaction_name, reaction_params)

Reacción 1 - Concentraciones finales (kg/s):
  C: 0.3901 kg/s
  CO2: 0.2852 kg/s
  CO: 0.1831 kg/s
Reacción 2 - Concentraciones finales (kg/s):
  C: 0.1454 kg/s
  H2O: 0.1171 kg/s
  CO: 0.7532 kg/s
  H2: -0.0280 kg/s
Reacción 3 - Concentraciones finales (kg/s):
  C: 0.3896 kg/s
  H2: 0.0129 kg/s
  CH4: 0.1052 kg/s
